In [1]:
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [2]:
train_dataset_root_path = "./data/cifar10/train"
random_seed = 420
torch.manual_seed(random_seed)
batch_size = 128

# Loading the dataset
In the explore_dataset.ipynb we saw that the images of cifar are RGB images, so each image has a dimension of 32 * 32 * 3 (H * W * C).
Now we load the the training dataset with help of the ImageFolder data loader, that is specific to dataset folder structures like cifar.
By providing ToTensor as transform, images are transformed from PIL.Image.Image objects to tensors [C * H * W] and values are rescaled from [0,255] to [0.0, 1.0] 

In [3]:
train_dataset = ImageFolder(
    root=train_dataset_root_path,
    transform=ToTensor()
)

ImageFolder inherits from VisionDataset, so it should be compatible with further torchvsion functions.  
As we can see by inspecting the variable train_dataset, the ImageFolder or VisionDataset does not load all images at once, but creates an index structure over the existing folder structure (a mapping of index to image location on disk)
# Splitting the dataset
We set aside some of the training samples for validation. Given the VisionDataset the random_split function creates two new objects of type Subset. A subset consists of a reference to the orginal VisinonDataset aka ImageFolder and a sublist of the indices.

In [4]:
validation_dataset_size = 5000
train_dataset_size = len(train_dataset) - validation_dataset_size

train_subset, validation_subset = random_split(train_dataset, [train_dataset_size, validation_dataset_size])

# DataLoaders
DataLoaders are yet another layer on top of the Subsets we created. Each dataloader combines a sampling method, e.g. randomly picking samples, with a dataset so that we can easily draw 
batches of samples.

In [5]:
train_data_loader = DataLoader(
    dataset=train_subset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=False  # should be True if cuda is available
)
validation_data_loader = DataLoader(
    dataset=validation_subset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=False  # should be True if cuda is available
)

# Constructing the CNN Network

In [6]:
from models import Cifar10CnnModel, accuracy, evaluate, fit

In [7]:
model = Cifar10CnnModel()

In [8]:
evaluate(model, train_data_loader)

{'val_loss': 2.3028876781463623, 'val_acc': 0.10051688551902771}

=> without training 0.10051 accuracy makes sense with 10 classes

In [10]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.001

**Note**: I do not have CUDA available locally, so up to here nothing is done to shift load to the GPU. Should move to Colab and make use of GPU as training seems not feasible with my laptop CPU... (after 41 minutes not one of 10 epochs is done)

In [ ]:
history = fit(num_epochs, lr, model, train_data_loader, validation_data_loader, opt_func)

### Test Plain18Layer

In [16]:
import models
from importlib import reload
reload(models)
pl18_model = models.Plain18Layer()

In [17]:
evaluate(pl18_model, validation_data_loader)

{'val_loss': 2.302842855453491, 'val_acc': 0.09433593600988388}

In [ ]:
history = fit(num_epochs, lr, model, train_data_loader, validation_data_loader, opt_func)


# Load a model saved after traning in colab

In [12]:
from models import Cifar10CnnModel

model = Cifar10CnnModel()
model.network.load_state_dict(torch.load("custom_cnn/savedmodels/Cifar10CnnModel_20231128_12:26", map_location=torch.device('cpu')))

evaluate(model, validation_data_loader)

{'val_loss': 0.7971807718276978, 'val_acc': 0.73046875}

=> must use the pretrained params, but values are slightly of. Is this due to different machine or GPU for training and CPU for evaluation